<a href="https://colab.research.google.com/github/andrkech/GENERATIVE-METHODS-IN-GENOMICS/blob/main/FASTQ_Dataset_Builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Import XML files with FASTQ files' links.

In [ ]:
!pip install rarfile

import rarfile
import os

dataset_drive_path = "/content/drive/MyDrive/BIOINFORMATICS/THESIS_KECHAGIAS/DATA/ena_sra-experiment_20231030-2024.rar"
dataset_colab_path = "/content/fastq_dataset/"

os.makedirs(dataset_colab_path, exist_ok=True)

with rarfile.RarFile(dataset_drive_path) as rf:
  rf.extractall(dataset_colab_path)

## Download FASTQ files.

In [ ]:
from lxml import etree

# Create an XMLParser with error recovery
parser = etree.XMLParser(recover=True)

xml_file_path = "/content/fastq_dataset/ena_sra-experiment_20231030-2024.xml"

# Try to parse the XML file
try:
    tree = etree.parse(xml_file_path, parser=parser)
    root = tree.getroot()
except Exception as e:
    print("An error occurred while parsing the XML file:")
    print(e)
    root = None

# Download FASTQ files
fastq_links = []

if root is not None:
    for experiment in root.findall('.//EXPERIMENT'):
        experiment_links = experiment.findall('.//EXPERIMENT_LINKS/EXPERIMENT_LINK/XREF_LINK')
        for link in experiment_links:
            db = link.find('DB')
            if db is not None and db.text == 'ENA-FASTQ-FILES':
                fastq_link = link.find('ID')
                if fastq_link is not None:
                    fastq_links.append(fastq_link.text)

'''
# Print the list of FASTQ download links
for link in fastq_links:
    print(link)

print(len(fastq_links))

'''

fastq_links_file = "/content/fastq_links.txt"
with open(fastq_links_file, 'w') as file:
    file.write('\n'.join(fastq_links))

In [ ]:
import requests

# Download the FASTQ files
download_dir = "/content/fastq_files/"
os.makedirs(download_dir, exist_ok=True)

for link in fastq_links:
    file_name = link.split('/')[-1]
    file_path = os.path.join(download_dir, file_name)

    response = requests.get(link, stream=True)
    if response.status_code == 200:
        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                f.write(chunk)

print(f"Found {link_count} FASTQ links.")
print(f"Links are saved to {links_file_path}.")
print(f"FASTQ files are downloaded to {download_dir}.")